# DeepAR

## Import Packages

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam

import numpy as np
import os
import random
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import StandardScaler
import util
from datetime import date
import argparse
from torch.utils.tensorboard.writer import SummaryWriter

## Define Modules

In [ ]:
class Gaussian(nn.Module):

    def __init__(self, hidden_size, output_size):
        '''
        Gaussian Likelihood Supports Continuous Data
        Args:
        input_size (int): hidden h_{i,t} column size
        output_size (int): embedding size
        '''
        super(Gaussian, self).__init__()
        self.mu_layer = nn.Linear(hidden_size, output_size)
        self.sigma_layer = nn.Linear(hidden_size, output_size)

        # initialize weights
        # nn.init.xavier_uniform_(self.mu_layer.weight)
        # nn.init.xavier_uniform_(self.sigma_layer.weight)

    def forward(self, h):
        _, _, hidden_size = h.size()
        sigma_t = torch.log(1 + torch.exp(self.sigma_layer(h))) + 1e-6
        sigma_t = sigma_t.squeeze(0)
        mu_t = self.mu_layer(h).squeeze(0)
        return mu_t, sigma_t


class NegativeBinomial(nn.Module):

    def __init__(self, input_size, output_size):
        '''
        Negative Binomial Supports Positive Count Data
        Args:
        input_size (int): hidden h_{i,t} column size
        output_size (int): embedding size
        '''
        super(NegativeBinomial, self).__init__()
        self.mu_layer = nn.Linear(input_size, output_size)
        self.sigma_layer = nn.Linear(input_size, output_size)

    def forward(self, h):
        _, _, hidden_size = h.size()
        alpha_t = torch.log(1 + torch.exp(self.sigma_layer(h))) + 1e-6
        mu_t = torch.log(1 + torch.exp(self.mu_layer(h)))
        return mu_t, alpha_t

In [ ]:
def gaussian_sample(mu, sigma):
    '''
    Gaussian Sample
    Args:
    ytrue (array like)
    mu (array like)
    sigma (array like): standard deviation

    gaussian maximum likelihood using log 
        l_{G} (z|mu, sigma) = (2 * pi * sigma^2)^(-0.5) * exp(- (z - mu)^2 / (2 * sigma^2))
    '''
    ypred = mu + torch.randn(mu.size(), dtype=mu.dtype, device=mu.device) * sigma
    return ypred


def negative_binomial_sample(mu, alpha):
    '''
    Negative Binomial Sample
    Args:
    ytrue (array like)
    mu (array like)
    alpha (array like)

    maximuze log l_{nb} = log Gamma(z + 1/alpha) - log Gamma(z + 1) - log Gamma(1 / alpha)
                - 1 / alpha * log (1 + alpha * mu) + z * log (alpha * mu / (1 + alpha * mu))

    minimize loss = - log l_{nb}

    Note: torch.lgamma: log Gamma function
    '''
    var = mu + mu * mu * alpha
    ypred = mu + torch.randn(mu.size(), dtype=mu.dtype, device=mu.device) * torch.sqrt(var)
    return ypred

## Define Model

In [ ]:
class DeepAR(nn.Module):

    def __init__(self, feature_size, hidden_size,
                 output_size, num_layers, likelihood="g"):
        '''The DeepAR model
        
        Parameters
        ----------
        feature_size
            The number of features in the input
        hidden_size
            The size of the hidden vectors
        output_size
            The size of the output vectors
        num_layers
            The number of lstm layers
        likelihood
            The likelihood function to use:
                gaussian or negative binomial
        '''
        super(DeepAR, self).__init__()

        self.encoder = nn.LSTM(feature_size+output_size, hidden_size,
                               num_layers, bias=True, batch_first=True)
        if likelihood == "g":
            self.likelihood_layer = Gaussian(hidden_size, output_size)
        elif likelihood == "nb":
            self.likelihood_layer = NegativeBinomial(hidden_size, output_size)
        self.likelihood = likelihood

    def forward(self, Xp, yp, Xf):
        '''Forward Pass
        
        Parameters
        ----------
        Xp
            The past covariates
        yp
            The past output variables
        Xf
            The future covariates
            
        Returns
        -------
        Tuple
            
        '''
        batch_size, history, num_features = Xp.size()
        _, horizon, _ = Xf.size()
        _, _, num_outputs = yp.size()
        ypred = []
        mus = []
        sigmas = []
        h, c = None, None
        for s in range(history + horizon):
            if s == 0:
                yin = yp[:, s:s+1, :].new_zeros([batch_size, 1, num_outputs])
                xin = Xp[:, s:s+1, :]
            elif 0 < s < history:
                yin = yp[:, s-1:s, :]
                xin = Xp[:, s:s+1, :]
            elif s == history:
                yin = yp[:, s-1:s, :]
                xin = Xf[:, s-history:s-history+1, :]
            else:
                xin = Xf[:, s-history:s-history+1, :]
            
            # pass through the LSTM
            inp = torch.cat([xin, yin], dim=2)
            if h is None and c is None:
                out, (h, c) = self.encoder(inp)
            else:
                out, (h, c) = self.encoder(inp, (h, c))
            
            # last layer(ll) hidden state
            h_ll = h[-1:, :, :].permute(1, 0, 2)
            h_ll = F.relu(h_ll)
            mu, sigma = self.likelihood_layer(h_ll)
            mus.append(mu)
            sigmas.append(sigma)
            if self.likelihood == "g":
                yin = gaussian_sample(mu, sigma)
            elif self.likelihood == "nb":
                alpha_t = sigma
                mu_t = mu
                yin = negative_binomial_sample(mu_t, alpha_t)
            # if without true value, use prediction
            if s >= history and s < horizon + history:
                ypred.append(yin)
        ypred = torch.cat(ypred, dim=1)
        mu = torch.cat(mus, dim=1)
        sigma = torch.cat(sigmas, dim=1)
        return ypred, mu, sigma

## Define Hyper-parameters

In [ ]:
args = argparse.Namespace()

args.num_epochs = 1000
args.steps_per_epoch = 150
args.lr = 5e-3
args.n_layers = 3
args.hidden_size = 120
args.likelihood = "g"
args.horizon = 24
args.history = 24*7
args.standard_scaler = True
args.log_scaler = False
args.mean_scaler = False
args.batch_size = 512
args.sample_size = 100

## Pre-process Dataset

In [ ]:
def load_data():
    data_path = '../../Merged-update_hourly.csv'
    data = pd.read_csv(os.path.join(data_path), index_col=0)
    data.fillna(0, inplace=True)
    known_variables = data.loc[
        :, ['WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B',
            'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN']
    ]
    unknown_variables = data.loc[
        :, ['FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26',
            'HWS_S26']
    ]
    target_variables = data.loc[
        :, ['WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26']
    ]
    output_variables = pd.concat([target_variables, unknown_variables], axis=1)
    X = known_variables.to_numpy()
    y = output_variables.to_numpy()
    Xtr, ytr, Xte, yte = util.train_test_split(X, y)
    return Xtr, ytr, Xte, yte

In [ ]:
Xtr, ytr, Xte, yte = load_data()

In [ ]:
Xtr.shape

In [ ]:
Xte.shape

## Training

In [ ]:
def batch_generator(X, y, history, horizon, batch_size):
    '''Given the dataset, generate a batch for training
    
    Parameters
    ----------
    X
        The input variables
    y
        The output variables
    history
        The number of hours of history to input
    horizon
        The number of hours into the future to output
    batch_size
        The batch size for training
        
    Returns
    -------
    Tuple
        The 
    '''
    total_time, features = X.shape
    if total_time < batch_size:
        batch_size = total_time
    batch = random.sample(range(history, total_time - horizon), batch_size)
    Xp = np.stack([X[idx - history: idx, :] for idx in batch])
    yp = np.stack([y[idx - history: idx, :] for idx in batch])
    Xf = np.stack([X[idx: idx+horizon, :] for idx in batch])
    yf = np.stack([y[idx: idx+horizon, :] for idx in batch])
    return Xp, yp, Xf, yf

In [ ]:
def train(
    Xtr,
    ytr,
    args
):
    '''Train the model
    
    Parameters
    ----------
    Xtr
        The training dataset
    ytr
        The target dataset
    args
        The configuration
    '''
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    _, feature_size = Xtr.shape
    _, output_size = ytr.shape
    model = DeepAR(feature_size, args.hidden_size, output_size,
                   args.n_layers, args.likelihood)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)  # use all GPUs
    model.to(device)
    optimizer = Adam(model.parameters(), lr=args.lr)
    random.seed(2)

    yscaler = None
    if args.standard_scaler:
        yscaler = util.StandardScaler()
    elif args.log_scaler:
        yscaler = util.LogScaler()
    elif args.mean_scaler:
        yscaler = util.MeanScaler()
    if yscaler is not None:
        ytr = yscaler.fit_transform(ytr)

    # training
    writer = SummaryWriter()
    horizon = args.horizon
    history = args.history
    step = 0
    for epoch in tqdm(range(args.num_epochs)):
        tqdm.write("Epoch {} starts...".format(epoch))
        for step in range(args.steps_per_epoch):
            # generate batch
            Xp, yp, Xf, yf = batch_generator(
                Xtr, ytr, history, horizon, args.batch_size)
            Xp = torch.from_numpy(Xp).float().to(device)
            yp = torch.from_numpy(yp).float().to(device)
            Xf = torch.from_numpy(Xf).float().to(device)
            yf = torch.from_numpy(yf).float().to(device)
            
            # forward pass
            ypred, mu, sigma = model(Xp, yp, Xf)
            y = torch.cat([yp, yf], dim=1)
            if args.likelihood == "g":
                loss = util.gaussian_likelihood_loss(y, mu, sigma)
            elif args.likelihood == "nb":
                loss = util.negative_binomial_loss(y, mu, sigma)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            writer.add_scalar('Train/Loss', loss, step)

In [ ]:
train(Xtr, ytr, args)

## Testing